<a href="https://colab.research.google.com/github/fezzibasma/Auto-drive/blob/main/Auto_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Auto-drive using Facebook AI Research's next generation library that provides state-of-the-art detection and segmentation algorithms.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 29.6 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44090 sha256=515a0fb3aebb34af16ff89e566009b954b27bb3e758cf7e0254e26741d72f096
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Cloning into 'detectron2'...
remote: Enumerating objects: 14608, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects:

In [ ]:
import torch
import detectron2
import torchvision
import cv2

## Register the data to Detectron2 config

In [ ]:
#Import librairies
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow 
%matplotlib inline

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer,ColorMode
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog, DatasetCatalog

## Prepare the dataset 

In [ ]:
mv /content/drive/MyDrive/RoadSign/Archive.zip /content/Archive.zip

In [ ]:
# download, decompress the data
!wget /content/Archive.zip
!unzip Archive.zip > /dev/null

/content/Archive.zip: Scheme missing.


In [ ]:
%pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Convert xml into json file

In [ ]:
import json
import xmltodict

path="/content/Archive/train"
for filename in os.listdir(path):
    if not filename.endswith('.xml'):
        continue

    fullname = os.path.join(path, filename)

    with open(fullname, 'r') as f:
        xmlString = f.read()

    jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)

    with open(fullname[:-4] + ".json", 'w') as f:
        f.write(jsonString)

In [ ]:
import json
import xmltodict

path="/content/Archive/val"
for filename in os.listdir(path):
    if not filename.endswith('.xml'):
        continue

    fullname = os.path.join(path, filename)

    with open(fullname, 'r') as f:
        xmlString = f.read()

    jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)

    with open(fullname[:-4] + ".json", 'w') as f:
        f.write(jsonString)

## Resize image for training

In [ ]:
#!/usr/bin/python
from PIL import Image
import os, sys

path = "/content/Archive/train/"
dirs = os.listdir( path )

def resize():
  for item in dirs:
    if item.endswith(".png") :
      if os.path.isfile(path+item) and item.endswith(".png"):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((200,200), Image.ANTIALIAS)
            imResize.save(path+item)
resize()

In [ ]:
#!/usr/bin/python
from PIL import Image
import os, sys

path = "/content/Archive/val/"
dirs = os.listdir( path )

def resize():
    for item in dirs:
      if item.endswith(".png") :
        if os.path.isfile(path+item) and item.endswith(".png"):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((200,200), Image.ANTIALIAS)
            imResize.save(path+item)

resize()

In [ ]:
for file in os.listdir("/content/Archive/train") :
  if file.endswith(".png") :
    print(file)

In [ ]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
      
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["annotation"]["filename"])
        #filename = os.listdir(directory) 
        
        record["file_name"] = filename
        record["height"] = img_anns["annotation"]["size"]["height"]
        record["width"] = img_anns["annotation"]["size"]["width"]
        record["depth"] = img_anns["annotation"]["size"]["depth"]

        annos = img_anns["annotation"]["object"]
        objs = []
        for anno in annos:
        #     px = [a[0] for a in anno['points']] # x coord
        #     py = [a[1] for a in anno['points']] # y-coord
        #     poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
        #     poly = [p for x in poly for p in x]

            obj = {
                # "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox": np.asarray([annos["bndbox"]["xmin"], annos["bndbox"]["ymin"], annos["bndbox"]["xmax"], annos["bndbox"]["ymax"]],dtype=int),
                "bbox_mode": BoxMode.XYXY_ABS,
                # "segmentation": [poly],
                "segmentation": [img_anns["annotation"]["segmented"]],
                "category_id": classes.index(annos["name"]),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:

import json
  
# Opening JSON file
f = open('/content/Archive/train/road0.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [ ]:
data["annotation"]["object"]["bndbox"]["xmin"]

'98'

In [ ]:
classes = ['speedlimit', 'crosswalk','stop','trafficlight']

data_path = '/content/Archive/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "cat_"+d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )

    MetadataCatalog.get("cat_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("cat_train")

AssertionError: ignored

## Training model with Instance Segmentation 

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cat_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00000
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

In [ ]:
# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
# trainer.resume_or_load(resume=False)

[11/24 23:05:40 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

TypeError: ignored